# wis-stac数据流及代码测试

## 从minio读数据

如era5、gpm

## 从stac读数据

如landsat、dem等

### step 1：加载测试数据，获取数据范围

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.read_file('test.json')
gdf

In [ ]:
from leafmap import Map

In [ ]:
m = Map()

In [ ]:
style = {
    "stroke": True,
    "color": "#CC0000 ",
    "weight": 2,
    "opacity": 1,
    "fill": False,
}
m.add_gdf(gdf,layer_name='test layer',style=style)
m

In [ ]:
geom = gdf.geometry[0]
geom

In [ ]:
geom.geom_type

In [ ]:
x,y = geom.exterior.xy
x,y

In [ ]:
[[i,j] for i,j in zip(x,y) ]

In [ ]:
# 构造数据范围的dictionary，方便后面在stac中search
bbox_latlon = {
    "type":f"{geom.geom_type}",
    "coordinates":[[[i,j] for i,j in zip(x,y) ]]
}
bbox_latlon

### step 2：通过stac搜索数据，获取items

如果获取到items，或items的len大于0，则说明通过测试数据范围能够搜索到landsat数据，反之该范围内没有landsat数据

In [ ]:
import planetary_computer as pc
import pystac_client

In [ ]:
start_date = "2021-06-30"  # Start date of the cube
end_date = "2021-07-01"  # End date of the cube

# aws
stac = "https://earth-search.aws.element84.com/v0"
collection = "landsat-8-l1-c1"  # Name of the STAC collection

# microsoft planetary computer
# stac = "https://planetarycomputer.microsoft.com/api/stac/v1"
# collection = "landsat-c2-l2"  # Name of the STAC collection

In [ ]:
# Open the Catalogue
CATALOG = pystac_client.Client.open(stac)

In [ ]:
# Do a search
SEARCH = CATALOG.search(
    intersects=bbox_latlon,
    datetime=f"{start_date}/{end_date}",
    collections=[collection],
    # **kwargs,
)

In [ ]:
# Get all items and sign if using Planetary Computer
items = SEARCH.get_all_items()
if stac == "https://planetarycomputer.microsoft.com/api/stac/v1":
    items = pc.sign(items)
items

In [ ]:
len(items)

### step 3：测试1条item，获取其范围

In [ ]:
item = items[0]
item

In [ ]:
item.id

In [ ]:
item.bbox

In [ ]:
print(item.datetime)

In [ ]:
item.geometry

In [ ]:
item.properties

In [ ]:
from shapely.geometry import shape
# from shapely.geometry.polygon import Polygon

In [ ]:
shape(item.geometry).wkt

In [ ]:
import pandas as pd

In [ ]:
# 构造一个dataframe，共三列，第一列是遥感影像数据的id，第二列是属性，第三列是遥感影像的范围
df = pd.DataFrame(
    {
        'id':[item.id],
        'properties':[item.properties],
        'geometry':[shape(item.geometry).wkt]
    }
)
df

In [ ]:
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
df

In [ ]:
lgdf = gpd.GeoDataFrame(df, geometry='geometry',crs='EPSG:4326')
lgdf

In [ ]:
lgdf.plot()

In [ ]:
style = {
    "stroke": True,
    "color": "#0000ff",
    "weight": 2,
    "opacity": 1,
    "fill": False,
    "fillColor": "#0000ff",
    "fillOpacity": 0.1,
}
m.add_gdf(lgdf,layer_name=item.id,style=style)

In [ ]:
m

### step 4：把搜索到的items范围都加上来

In [ ]:
ids = []
properties = []
geometries = []
for it in items:
    ids.append(it.id)
    properties.append(it.properties)
    geometries.append(shape(it.geometry).wkt)

In [ ]:
dfs = pd.DataFrame(
    {
        'id': ids,
        'properties': properties,
        'geometry': geometries
    }
)
dfs

In [ ]:
dfs['geometry'] = gpd.GeoSeries.from_wkt(dfs['geometry'])
dfs

In [ ]:
gdfs = gpd.GeoDataFrame(dfs, geometry='geometry',crs='EPSG:4326')
gdfs

In [ ]:
m.add_gdf(gdfs,layer_name=collection,style=style)
m